In [1]:
import numpy as np
import pandas as pd

**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [2]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [3]:
city_reg_map = {k:v for k,v in np.concatenate([[(v,k) for v in vs] for k,vs in geo_data.items()])}
city_reg_map

{'владивосток': 'Дальний Восток',
 'москва': 'Центр',
 'мурманск': 'Северо-Запад',
 'петербург': 'Северо-Запад',
 'псков': 'Северо-Запад',
 'сахалин': 'Дальний Восток',
 'тула': 'Центр',
 'хабаровск': 'Дальний Восток',
 'ярославль': 'Центр'}

In [4]:
keywords_ds = pd.read_csv('keywords.csv')
keywords_ds.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [5]:
def get_region(s):
    for city in city_reg_map:
        if city in s:
            return city
    return 'undefined'

In [6]:
keywords_ds['region'] = keywords_ds['keyword'].apply(lambda x: get_region(x))

In [7]:
keywords_ds[keywords_ds['region']!='undefined'].head()

,keyword,shows,region
127,авито москва,979292,москва
370,авито ру санкт петербург,425134,петербург
564,погода в санкт петербурге,292489,петербург
849,авито ярославль,209581,ярославль
1063,фарпост владивосток,176951,владивосток


**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [8]:
movies_ds = pd.read_csv('ml-latest-small/movies.csv')
ratings_ds = pd.read_csv('ml-latest-small/ratings.csv')

In [9]:
mean_rate = ratings_ds.groupby('movieId')['rating'].mean().to_dict()
movies_ds['mean_rating'] = movies_ds['movieId'].apply(lambda x: mean_rate.get(x,None))

In [10]:
movies_ds['class'] = movies_ds['mean_rating'].apply(lambda x: None if x is None else "высокий" if x>=4.5 else "средний" if x>2 else "низкий")

**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [11]:
users_ds = pd.DataFrame()
users_ds['userId'] = ratings_ds['userId'].drop_duplicates().sort_values().values
users_ds['n_ratings'] = users_ds['userId'].apply(lambda id_: ratings_ds[ratings_ds['userId']==id_]['rating'].count())
users_ds['min_timestamp'] = users_ds['userId'].apply(lambda id_: ratings_ds[ratings_ds['userId']==id_]['timestamp'].min())
users_ds['max_timestamp'] = users_ds['userId'].apply(lambda id_: ratings_ds[ratings_ds['userId']==id_]['timestamp'].max())
users_ds['lifetime'] = users_ds['max_timestamp']-users_ds['min_timestamp']
users_ds['lifetime_days'] = users_ds['lifetime']//3600//24

In [12]:
users_ds[users_ds['n_ratings']>=100]['lifetime_days'].mean()

511.06854838709677

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [13]:
years = np.arange(1950, 2011)

In [14]:
movies_ds.head()

,movieId,title,genres,mean_rating,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930,средний
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818,средний
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615,средний
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143,средний
4,5,Father of the Bride Part II (1995),Comedy,3.071429,средний


In [15]:
def production_year(title):
    for y in years:
        if str(y) in title:
            return y
    return 1900

In [16]:
movies_ds['year'] = movies_ds['title'].apply(lambda x: production_year(x))

In [17]:
movies_ds.groupby('year')['mean_rating'].mean().sort_values(ascending=False).head()

year
1971    3.811243
1957    3.767680
1951    3.718044
1952    3.700104
1955    3.672863
Name: mean_rating, dtype: float64